# 

In [ ]:
# ToDo: Elegir columnas de dataframe condicionalmente

In [1]:
#### Decompress
import zipfile
import tarfile 
import os
import sys
import errno


#### Json to dataframe

import json
import pandas as pd
import pyarrow.parquet as pq

In [2]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
err_logger = [];
compressed_dir="./data/4/"
json_dir="./data/4_json/"
parquet_dir="./data/4_parquet/"
parquet_trim_dir="./data/4_parquet_trim/"
csv_trim_dir="./data/4_csv_trim/"

# Pueden agregar todas las columnas que les sean relevantes
relevant_cols = ["extended_tweet.full_text", "user.verified", "user.followers_count", 
                 "user.friends_count", "quote_count", "reply_count", "retweet_count", 
                 "favorite_count", "lang", "entities.hashtags", "entities.urls", "geo.coordinates"]



In [5]:
def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

make_sure_path_exists(compressed_dir)
make_sure_path_exists(json_dir)
make_sure_path_exists(parquet_dir)
make_sure_path_exists(parquet_trim_dir)
make_sure_path_exists(csv_trim_dir)

Extraer archivos

In [6]:

for file in os.listdir(compressed_dir):
    print("Processing " + file)
    try:
        if file.endswith('.zip'):
            with zipfile.ZipFile(compressed_dir+file, 'r') as zip_ref:
                zip_ref.extractall(json_dir)
        elif file.endswith('tar.gz'):
            with tarfile.open(compressed_dir+file) as file:
                file.extractall(json_dir)
        else:
            err_logger.append({"operation": "Process compressed", "detail": compressed_dir + file, "err":"Unrecognized file"})
    except Exception as e:
        err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        err_message = err_template.format(type(e).__name__, e.args)
        err_logger.append({"operation":"Extract", "detail": compressed_dir + file, "err":err_message})
        continue


Processing twitter-2021.07.2.tar.gz
Processing twitter-2021.07.7.tar.gz
Processing twitter-2021.07.19.tar.gz
Processing twitter-2021.07.8.tar.gz
Processing twitter-2021.07.18.tar.gz
Processing twitter-2021.07.15.tar.gz
Processing twitter-2021.07.4.tar.gz
Processing twitter-2021.07.1.tar.gz
Processing twitter-2021.07.3.tar.gz
Processing twitter-2021.07.10.tar.gz
Processing twitter-2021.07.16.tar.gz
Processing twitter-2021.07.11.tar.gz
Processing twitter-2021.07.12.tar.gz
Processing twitter-2021.07.13.tar.gz
Processing twitter-2021.07.5.tar.gz
Processing twitter-2021.07.9.tar.gz
Processing twitter-2021.07.14.tar.gz
Processing twitter-2021.07.20.tar.gz
Processing twitter-2021.07.22.tar.gz
Processing twitter-2021.07.21.tar.gz
Processing twitter-2021.07.24.tar.gz
Processing twitter-2021.07.23.tar.gz
Processing twitter-2021.07.17.tar.gz
Processing twitter-2021.07.6.tar.gz
Processing twitter-2021.07.28.tar.gz
Processing twitter-2021.07.31.tar.gz
Processing twitter-2021.07.26.tar.gz
Processing

In [3]:
import json
import pandas as pd
import pyarrow.parquet as pq

In [8]:
#t = pd.read_json(json_dir+"twitter-2021.07.10.json", lines = True, orient='records')
#print(t.shape)
#list(t)
#t.extended_tweet[0]

In [4]:
import json
for file in os.listdir(json_dir):
    print("Processing " + file)
    data = []
    
    try:
        with open(json_dir+file) as f:
            for line in f:
                data.append(json.loads(line))

    except Exception as e: 
        err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        err_message = err_template.format(type(e).__name__, e.args)
        err_logger.append({"operation":"Load json", "detail":json_dir + file, "err":err_message})
        continue
        
    try:
        # Json to dataframe
        df = pd.json_normalize(data)
        
        # Guardar dataframe completo
        #df.to_parquet(parquet_dir+file+".parquet", engine='auto', compression='snappy')
        
        # Seleccionar columnas
        df_trim = df.loc[:, relevant_cols]
        
        # Guardar dataframe reducido
        df_trim.to_parquet(parquet_trim_dir+file+".parquet", engine='auto', compression='snappy')
        
        # Guardar en csv
        #df.to_csv(csv_trim_dir+file+".csv", sep=',', encoding='utf-8')
    except Exception as e: 
        err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        err_message = err_template.format(type(e).__name__, e.args)
        err_logger.append({"operation":"Dataframe & parquet", "detail":json_dir + file, "err":err_message})
        continue

Processing twitter-2021.07.2.json


In [ ]:
'''
# Alta demanda de RAM, use con precaución

import json

data = []

for file in os.listdir(json_dir):
    print("Processing " + file)
    
    try:
        with open(json_dir+file) as f:
            for line in f:
                data.append(json.loads(line))

    except Exception as e: 
        err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        err_message = err_template.format(type(e).__name__, e.args)
        err_logger.append({"operation":"Load json", "detail":json_dir + file, "err":err_message})
        continue
    
try:
    # Json to dataframe
    df = pd.json_normalize(data)
    df.to_parquet(parquet_dir+"all.parquet", engine='auto', compression='snappy')
    df_trim = df.loc[:, relevant_cols]
    df_trim.to_parquet(parquet_trim_dir+"all.parquet", engine='auto', compression='snappy')
except Exception as e: 
    err_template = "An exception of type {0} occurred. Arguments:\n{1!r}"
    err_message = err_template.format(type(e).__name__, e.args)
    err_logger.append({"operation":"Dataframe & parquet", "detail":json_dir + "all.parquet", "err":err_message})

'''

In [5]:
# Mostrar todos los errores del proceso
print(err_logger)

[{'operation': 'Load json', 'detail': './data/4_json_fail/twitter-2021.07.2.json', 'err': 'An exception of type JSONDecodeError occurred. Arguments:\n("Expecting \',\' delimiter: line 2 column 1 (char 2737)",)'}]


In [11]:
# Cargar alguno de los parquet generados
df2 = pd.read_parquet(parquet_trim_dir+"twitter-2021.07.10.json.parquet")
df2.shape

(87280, 12)

In [12]:
df2.head()

,extended_tweet.full_text,user.verified,user.followers_count,user.friends_count,quote_count,reply_count,retweet_count,favorite_count,lang,entities.hashtags,entities.urls,geo.coordinates
0,Aunque sea para la foto jejeje...\n@monitaf_31...,False,459,503,0,0,0,0,es,"[{'indices': [46, 55], 'text': 'relaxday'}, {'...",[{'display_url': 'twitter.com/i/web/status/1…'...,"[14.6405587, -90.7745264]"
1,None,False,1400,368,0,0,0,0,en,[],[],None
2,None,False,5,274,0,0,0,0,pt,[],[],None
3,None,False,9122,82,0,0,0,0,es,[],[],None
4,None,True,12456,2070,0,0,0,0,es,[],[],None


In [13]:
list(df2)

['extended_tweet.full_text',
 'user.verified',
 'user.followers_count',
 'user.friends_count',
 'quote_count',
 'reply_count',
 'retweet_count',
 'favorite_count',
 'lang',
 'entities.hashtags',
 'entities.urls',
 'geo.coordinates']